In [1]:
'''Importing necessary classes
'''
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os 
import math 

#os.chdir(os.getcwd() + '/camera_cal/' )

'''Loading images using glob API, converting to grayscale 
and finding corners in all images and returning image 
points and object points 
'''
def load_testImages(ncx,ncy):
    
    images = glob.glob('calibration*.jpg')
    #print('Size of input calibraion images list is ',len(images))
    image_list = []
    image_gray = [] 
    objpts = []
    imgpts = []
    obj = np.zeros((ncx*ncy,3), np.float32)
    obj[:,:2] = np.mgrid[0:ncx,0:ncy].T.reshape(-1,2) 
    
    for fname in images:    
        image = mpimg.imread(fname)
        image_list.append(image)
        gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        image_gray.append(gray)
        ret,corners = cv2.findChessboardCorners(gray,(ncx,ncy),None)
    
        if ret == True:
            objpts.append(obj)
            imgpts.append(corners)
    return objpts,imgpts,image_list,image_gray


def camera_calibration(objpts,imgpts,shape):
    ret,mat,dist,rv,lv = cv2.calibrateCamera(objpts,imgpts,shape,None,None)
    return ret,mat,dist,rv,lv

def undistort_image(img,mat,dist):
    undist = cv2.undistort(img,mat,dist,None,mat)
    return undist

def binary_gradient(img,clr_thresh,gradient_thresh,sobel_kernel):
    color1_up = clr_thresh[1]
    color1_down = clr_thresh[0]
    #print(len(clr_thresh))
    #color2_up = clr_thresh[3]
    #color2_down = clr_thresh[2]
    clr_y_dn = clr_thresh[2]
    clr_y_up = clr_thresh[3]
    clr_w_dn = clr_thresh[4]
    clr_w_up = clr_thresh[5]
    #print(clr_thresh)
    grad_up = gradient_thresh[1]
    grad_down = gradient_thresh[0]
    gradient_high = gradient_thresh[3]
    gradient_low = gradient_thresh[2]
    
    grad_xup = gradient_thresh[5]
    grad_xdown = gradient_thresh[4]
    grad_yup = gradient_thresh[7]
    grad_ydown = gradient_thresh[6]
    imshape = img.shape
    image2 = np.copy(img) 
    image1 = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    binary_hls  = np.zeros([imshape[0],imshape[1]])
    binary_white = np.zeros([imshape[0],imshape[1]])
    binary_yellow = np.zeros([imshape[0],imshape[1]])
    mask1 = cv2.inRange(image1,clr_y_dn,clr_y_up)
    mask2 = cv2.inRange(image1,clr_w_dn,clr_w_up)
    s_channel = image1[:,:,2]
    binary_white[mask2.nonzero()] = 255
    binary_yellow[mask1.nonzero()] = 255
    binary_hls[(s_channel<color1_up) & (s_channel>color1_down)] = 255
    #binary_white[mask2.nonzero()] = 255
    #cbinary = cv2.bitwise_or(binary_white,binary_hls)
    luv_image = cv2.cvtColor(image2,cv2.COLOR_RGB2Luv)
    l_mask = luv_image[:,:,0]
    l_binary = np.zeros([imshape[0],imshape[1]])
    l_binary[(l_mask<255) & (l_mask>200)] = 255
    
    lab_image = cv2.cvtColor(image2,cv2.COLOR_RGB2Lab)
    lab_mask = lab_image[:,:,0]
    lab_binary = np.zeros([imshape[0],imshape[1]])
    lab_binary[(lab_mask<255) & (lab_mask>200)] = 255
    
    
    
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    abs_sx = np.absolute(sobelx)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    abs_sy = np.absolute(sobely)
    sobel_mag = np.sqrt(np.square(sobelx) + np.square(sobely))
    sobel_mag = np.uint8(255*sobel_mag/np.max(sobel_mag))
    abs_x = np.absolute(sobelx)
    abs_y = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad = np.arctan2(abs_y, abs_x)
    
    
    sbinary = np.zeros([imshape[0],imshape[1]])
    #sbinary[((sobel_mag<=grad_up) & (sobel_mag>=grad_down) & (grad<gradient_high) & (grad>gradient_low))
    #       |( (abs_sx<grad_xup) & (abs_sy>grad_ydown) & (abs_sy>grad_ydown) & (abs_sy<grad_yup))] = 255
    sbinary[(sobel_mag<=grad_up) & (sobel_mag>=grad_down) & (grad<gradient_high) & (grad>gradient_low)] = 255
    bimg = np.zeros([imshape[0],imshape[1]])
    #binary = np.dstack(( np.zeros_like(sbinary), sbinary, binary_hls)) #binary = np.dstack((sbinary,binary_hls,bimg))
    #binary = cv2.bitwise_or(sbinary,binary_hls)
    
    binary = cv2.bitwise_or(sbinary,l_binary)
    binary = cv2.bitwise_or(binary,binary_hls)
    binary = cv2.bitwise_or(binary,lab_binary)
    #cbinary = cv2.bitwise_or(binary_yellow,binary_white) #commented 
    #binary = cv2.bitwise_or(cbinary,binary)  #commented
    #binary = np.dstack((binary1,np.zeros_like(sbinary),np.zeros_like(sbinary)))
    #plt.imshow(sbinary,cmap='gray')
    #plt.imshow(binary_hls,cmap='gray')
    #plt.imshow(binary)
    return binary

def lane_recognition(binary_warped,nwindows,margin,minipix,imshape):
    nonzeroy = np.array(binary_warped.nonzero()[0])
    nonzerox = np.array(binary_warped.nonzero()[1])
    hist = np.sum(binary_warped,axis = 0)
    left_inds = []
    right_inds = []
    #plt.plot(hist)
    centerx = int(imshape[0]/2)
    current_basel  = np.argmax(hist[0:centerx])
    current_baser  = np.argmax(hist[centerx:]) + centerx
    init_basel = current_basel
    init_baser = current_baser
    window_height = np.int(imshape[1]//nwindows)

    #print('Initial box centers are ',current_basel,current_baser,centerx)

    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) #np.copy(binary_warped) #
    #plt.imshow(binary_warped,cmap='gray')
    #plt.imshow(out_img)
    #print('This is the size of the dstack image ',out_img.shape)
    for window in range(nwindows) :

        ylow = imshape[1]- window*window_height
        yhigh = imshape[1]- (window+1)*window_height
        xlow_left = current_basel - margin
        xhigh_left = current_basel + margin
        xlow_right = current_baser - margin
        xhigh_right = current_baser + margin

        indsl = ((nonzerox<=xhigh_left) & (nonzerox>=xlow_left) & (nonzeroy>=yhigh) &(nonzeroy<=ylow)).nonzero()[0]
        indsr = ((nonzerox<=xhigh_right) & (nonzerox>=xlow_right) & (nonzeroy>=yhigh) &(nonzeroy<=ylow)).nonzero()[0]
        cv2.rectangle(out_img,(xlow_left,ylow),
        (xhigh_left,yhigh),(0,255,0), 2) 
        cv2.rectangle(out_img,(xlow_right,ylow),
        (xhigh_right,yhigh),(0,255,0), 2) 

        if(len(indsl)>minipix):
            current_basel = int(np.mean(nonzerox[indsl]))
        if(len(indsr)>minipix):
            current_baser = int(np.mean(nonzerox[indsr]))

        left_inds.append(indsl)
        right_inds.append(indsr)
        
    left_inds = np.concatenate(left_inds)
    right_inds = np.concatenate(right_inds)

    leftx = nonzerox[left_inds]
    lefty = nonzeroy[left_inds]
    rightx = nonzerox[right_inds]
    righty = nonzeroy[right_inds]
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    return leftx,lefty,rightx,righty,out_img,init_basel,init_baser

def lane_recognition2(binary_warped,nwindows,margin,minipix,imshape,poly_left,left_fit,poly_right,right_fit,ploty,last_width):
    nonzeroy = np.array(binary_warped.nonzero()[0])
    nonzerox = np.array(binary_warped.nonzero()[1])
    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) #np.copy(binary_warped) #
    left_inds = []
    right_inds = []
    for i in range(imshape[1]):
        y = i
        x_l = left_fit[0]*y**2 + left_fit[1]*y + left_fit[2]
        x_r = right_fit[0]*y**2 + right_fit[1]*y + right_fit[2]
        xlow_left =  x_l - margin
        xhigh_left = x_l + margin
        xlow_right = x_r - margin
        xhigh_right = x_r + margin
        indsl = ((nonzerox<=xhigh_left) & (nonzerox>=xlow_left) & (nonzeroy==y)).nonzero()[0]
        indsr = ((nonzerox<=xhigh_right) & (nonzerox>=xlow_right) & (nonzeroy==y)).nonzero()[0]
        #if(len(indsl)==0 ):
         #   indsl = #((nonzerox = x_l) & (nonzeroy ==y))
        left_inds.append(indsl)
        right_inds.append(indsr)
        
    left_inds = np.concatenate(left_inds)
    right_inds = np.concatenate(right_inds)
    leftx = nonzerox[left_inds]
    lefty = nonzeroy[left_inds]
    rightx = nonzerox[right_inds]
    righty = nonzeroy[right_inds]
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    return leftx,lefty,rightx,righty,out_img


def polynomial_fitting(leftx,lefty,rightx,righty,ploty,xm_per_pix,ym_per_pix):
    plotym = ploty*ym_per_pix
    leftym = lefty*ym_per_pix
    rightxm = rightx*xm_per_pix
    leftxm = leftx*xm_per_pix
    rightym = righty*ym_per_pix
    
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)

    left_fitm = np.polyfit(leftym,leftxm,2)
    right_fitm = np.polyfit(rightym,rightxm,2)
    
    try: 
        poly_left = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        poly_right = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        poly_leftm = left_fitm[0]*plotym**2 + left_fitm[1]*plotym + left_fitm[2]
        poly_rightm = right_fitm[0]*plotym**2 + right_fitm[1]*plotym + right_fitm[2]
        found_condition = True
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        found_condition = False
        poly_left = 1*ploty**2 + 1*ploty
        poly_right = 1*ploty**2 + 1*ploty
        poly_leftm = 1*plotym**2 + 1*plotym
        poly_rightm = 1*plotym**2 + 1*plotym
        
        
    return poly_left,poly_right,poly_leftm,poly_rightm,left_fit,right_fit,left_fitm,right_fitm,found_condition

def curvature_finding(ploty,poly_left,poly_right):
    y_eval = np.min(ploty)
    left_curverad = ((1+(2*poly_left[0]*y_eval + poly_left[1])**2)**(3/2))/(2*poly_left[0])
    right_curverad = ((1+(2*poly_right[0]*y_eval + poly_right[1])**2)**(3/2))/(2*poly_right[0]) 
    print(y_eval,poly_left,poly_right)
    #ploty1 = ploty*ym_per_pix
    #leftx1 = leftx*xm_per_pix
    #rightx1 = rightx*xm_per_pix
    #lefty1 = lefty*ym_per_pix
    #righty1 = righty*ym_per_pix
    #left_fit_cr = np.polyfit(lefty1, leftx1, 2)
    #right_fit_cr = np.polyfit(righty1, rightx1, 2)

    #y_eval = np.max(ploty1)
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    #left_curveradm = ((1+(2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**(3/2))/(2*left_fit_cr[0])#0  ## Implement the calculation of the left line here
    #right_curveradm = ((1+(2*right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**(3/2))/(2*right_fit_cr[0])  ## Implement the calculation of the right line here
    #print('Left and Right radii of curvature in metres are ',left_curveradm, right_curveradm)
    return left_curverad,right_curverad

def unwarp_image(binary_warped,Minv,imshape):
    unwarped = cv2.warpPerspective(binary_warped,Minv,imshape,None)
    return unwarped

class Lane_data():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  
        self.last_width = None

In [2]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from PIL import Image, ImageDraw,ImageFont
from functools import reduce

def process_image(image):
    
    #M = cv2.getPerspectiveTransform(src,dst)
    #Minv = cv2.getPerspectiveTransform(dst,src)
    M = np.array([[ -7.22021661e-01,  -1.53068592e+00,   1.10209386e+03],
                 [ -1.49880108e-15,  -2.07942238e+00,   9.77328520e+02],
                 [ -2.60208521e-18,  -2.39169675e-03,   1.00000000e+00]])
       
        
    Minv = np.array([[  1.71875000e-01,  -7.36111111e-01,   5.30000000e+02],
                     [  1.15463195e-14,  -4.80902778e-01,   4.70000000e+02],
                     [  1.97324795e-17,  -1.15017361e-03,   1.00000000e+00]])
        
        
    mat = np.array([[  1.15777818e+03,   0.00000000e+00,   6.67113857e+02],
                     [  0.00000000e+00,   1.15282217e+03,   3.86124583e+02],
                     [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])
    dist = np.array([[-0.24688507, -0.02373155, -0.00109831,  0.00035107, -0.00259867]])
    
    imshape = image.shape
    undistimage = undistort_image(image,mat,dist)
    new_img = cv2.warpPerspective(undistimage,M,(imshape[1],imshape[0]))

    clr_thresh = []
    clr_thresh.append(90)   #color thresholds
    clr_thresh.append(255)
    clr_thresh.append(np.array([15,0,100]))     #yellow
    clr_thresh.append(np.array([30,255,255]))
    clr_thresh.append(np.array([0,250,0]))       #white
    clr_thresh.append(np.array([179,255,255]))
    gradient_thresh = []
    gradient_thresh.append([30])
    gradient_thresh.append([255])
    gradient_thresh.append([-1.3])
    gradient_thresh.append([1.3])
    gradient_thresh.append([0])
    gradient_thresh.append([100])
    gradient_thresh.append([0])
    gradient_thresh.append([100])

    sobel_kernel = 5

    binary_image = binary_gradient(new_img,clr_thresh,gradient_thresh,sobel_kernel)

    #print('this is the size of binary warped image',binary_image.shape)
    binary_warped = np.copy(binary_image)

    nwindows = 9
    margin1 = 100
    margin2 = 20
    minipix = 50
    ploty = np.linspace(0,imshape[0]-1,imshape[0])
    #plt.imshow(binary_warped,cmap='gray')
    if(len(Left_lane.recent_xfitted)==0):
        leftx,lefty,rightx,righty,out_img,init_basel,init_baser = lane_recognition(binary_warped,nwindows,margin1,minipix,(imshape[1],imshape[0]))
    else:
        poly_left = Left_lane.bestx
        left_fit = Left_lane.current_fit
        poly_right = Right_lane.bestx
        right_fit = Right_lane.current_fit
        leftx,lefty,rightx,righty,out_img = lane_recognition2(binary_warped,nwindows,margin2,minipix,(imshape[1],imshape[0]),poly_left,left_fit,poly_right,right_fit,ploty,Left_lane.last_width)

    ym_per_pix = 15/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/1000 # (init_baser-init_basel) #meters per pixel in x dimension
    poly_left,poly_right,poly_leftm,poly_rightm,left_fit,right_fit,left_fitm,right_fitm,found_condition = polynomial_fitting(leftx,lefty,rightx,righty,ploty,xm_per_pix,ym_per_pix)
    
    minimum_length = np.absolute(np.min(poly_left-poly_right))
    print(minimum_length) 
    Left_lane.last_width = np.mean(poly_left-poly_right)
    Right_lane.last_width = np.mean(poly_left-poly_right)
    if(len(Left_lane.recent_xfitted)>0):
        crit0 = left_fit[0]/right_fit[0] #criteria #0 
        crit00 = np.absolute(crit0)
        if(minimum_length<700):
        #if((minimum_length<500) | (crit0<0)):
        #if((minimum_length<500)| (crit0<0) | ((crit0>0)& (crit00<5) & (crit00>1/5))):
            crit1 = np.absolute(np.divide((Left_lane.current_fit[:]-left_fit[:]),Left_lane.current_fit[:]))
            crit2 = np.absolute(np.divide((Right_lane.current_fit[:]-right_fit[:]),Right_lane.current_fit[:]))
            if((crit1[0]> 0.5) | (crit1[1]> 0.5) | (crit1[2]> 0.5)):
                poly_left = Left_lane.bestx
                left_fit = Left_lane.best_fit 
            if((crit2[0]> 0.5) | (crit2[1]> 0.5) | (crit2[2]> 0.5 )):  
                poly_right = Right_lane.bestx
                right_fit = Right_lane.best_fit     
                
            
    plotym = ploty*ym_per_pix
    left_curverad,right_curverad = curvature_finding(ploty,left_fit,right_fit)
    #print('Left and Right radii of curvature in pixels are ',left_curverad, right_curverad)
    left_curveradm,right_curveradm = curvature_finding(plotym,left_fitm,right_fitm)
    #print('Left and Right radii of curvature in meters are ',left_curveradm, right_curveradm)
    
    

            

        
    unwarped_image = unwarp_image(out_img,Minv,(imshape[1],imshape[0]))
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([poly_left, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([poly_right, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1],image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    #####################Text to write on image######################
    radius_mag = np.min(np.absolute(left_curveradm)+ np.absolute(right_curveradm)) #0.5*(np.absolute(left_curveradm)+ np.absolute(right_curveradm)) #
    end_y = image.shape[0]*ym_per_pix
    left_lanex = left_fitm[0]*end_y**2  + left_fitm[1]*end_y + left_fitm[2]
    right_lanex = right_fitm[0]*end_y**2  + right_fitm[1]*end_y + right_fitm[2]
    car_gap = round((0.5*(left_lanex+right_lanex) - image.shape[1]*xm_per_pix/2),2)

    if(car_gap> 0):
        side_car = 'The vehicle is ' + str(car_gap) + ' m to the left of center '
    elif(car_gap<0):
        side_car = 'The vehicle is ' + str(-car_gap) + ' m to the lright of center '
    else:
        side_car = 'The vehicle is at the center '

    radius_text = 'Radius of curvature is ' + str(np.int(radius_mag)) + ' (m)'
    print("This is the average radius of curvature of the anes " + str(radius_mag))
    #print(os.getcwd())
    #os.chdir('..')
    font_type = ImageFont.truetype("arial.ttf", 40, encoding="unic")
    #print(end_y,left_lanex,right_lanex,car_gap,side_car)
    #####################Text to write on image######################   
       #############################################################################################################       
    if(found_condition == True):
        numberl = len(Left_lane.recent_xfitted)
        #print(numberl)
        if(numberl == 0):
            Left_lane.recent_xfitted.append(poly_left)
            Left_lane.bestx = np.array(poly_left)
            Left_lane.best_fit = left_fit
            Left_lane.current_fit = left_fit
            Left_lane.radius_of_curvature = left_curverad
            Left_lane.line_base_pos = np.absolute(car_gap) 
            Left_lane.diffs = np.array([0,0,0], dtype='float') 
            Left_lane.allx = leftx
            Left_lane.ally = lefty
        else: 
            Left_lane.recent_xfitted.append(poly_left)
            Left_lane.diffs = Left_lane.current_fit-left_fit
            weights = np.ones(len(Left_lane.recent_xfitted))
            weights[0] = 0.8
            weights[1:] = 0.2/(len(weights) - 1)
            #Left_lane.bestx = reduce(lambda a,b: a + b[0]*b[1], zip(weights, Left_lane.recent_xfitted), np.zeros(len(Left_lane.bestx)))
            Left_lane.bestx = (Left_lane.bestx *(numberl) + poly_left)/(numberl +1 )
            Left_lane.best_fit = (Left_lane.best_fit *(numberl) + left_fit)/(numberl +1 )
            Left_lane.current_fit = left_fit
            Left_lane.radius_of_curvature = left_curverad
            Left_lane.line_base_pos = np.absolute(car_gap) 
            Left_lane.allx = leftx
            Left_lane.ally = lefty
         
        numberr = len(Right_lane.recent_xfitted)
        if(numberr == 0):
            Right_lane.recent_xfitted.append(poly_right)
            Right_lane.bestx = poly_right
            Right_lane.best_fit = right_fit
            Right_lane.current_fit = right_fit
            Right_lane.radius_of_curvature = right_curverad
            Right_lane.line_base_pos = np.absolute(car_gap) 
            Right_lane.diffs = np.array([0,0,0], dtype='float') 
            Right_lane.allx = rightx
            Right_lane.ally = righty
        else: 
            Right_lane.recent_xfitted.append(poly_right)
            Right_lane.diffs = Right_lane.current_fit-right_fit
            weights = np.ones(len(Right_lane.recent_xfitted))
            weights[0] = 0.8
            weights[1:] = 0.2/(len(weights) - 1)
            #Right_lane.bestx = reduce(lambda a,b: a + b[0]*b[1], zip(weights, Right_lane.recent_xfitted), np.zeros(len(Left_lane.bestx)))
            Right_lane.bestx = (Right_lane.bestx *(numberr) + poly_right)/(numberr +1 )
            Right_lane.best_fit = (Right_lane.best_fit *(numberr) + right_fit)/(numberr +1 )
            Right_lane.current_fit = right_fit
            Right_lane.radius_of_curvature = right_curverad
            Right_lane.line_base_pos = np.absolute(car_gap) 
            Right_lane.allx = rightx
            Right_lane.ally = righty
            
        
#############################################################################################################  
    result1 = np.copy(result)
    result = Image.fromarray(result) 
    draw = ImageDraw.Draw(result)
    draw.text(xy=(0,0),text = radius_text,fill = (255,0,0),font=font_type)
    draw.text(xy=(0,40),text = side_car,fill = (255,0,0),font=font_type)

    #if(numberl == 10):
     #   print(os.getcwd())
      #  mpimg.imsave('output_images/final_output.jpg',result)
    return np.asarray( result, dtype="int32" )  #np.asarray( result, dtype="int32" )
    #return np.dstack((binary_warped,binary_warped,binary_warped))
#np.dstack((binary_warped,binary_warped,binary_warped)) #
#np.dstack((binary_warped,binary_warped,binary_warped))



In [3]:
#print(os.getcwd())
#os.chdir('..')
white_output = 'output_images/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(2,5)
Left_lane = Lane_data()
Right_lane = Lane_data()
clip1 = VideoFileClip("project_video.mp4")#.subclip(45,50)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.ipython_display
%time white_clip.write_videofile(white_output, audio=False)

910.929639368
0.0 [ -1.59876641e-04   2.18080130e-01   2.10436132e+02] [ -1.74481171e-04   2.79547887e-01   1.08472043e+03]
0.0 [-0.00136292  0.03873103  0.77861369] [ -1.48741709e-03   4.96477047e-02   4.01346558e+00]
This is the average radius of curvature of the anes 705.082986625
[MoviePy] >>>> Building video output_images/project_video.mp4
[MoviePy] Writing video output_images/project_video.mp4


  0%|          | 1/1261 [00:00<13:29,  1.56it/s]

909.108655636
0.0 [ -1.55799739e-04   2.15317453e-01   2.10701431e+02] [ -1.87826224e-04   2.88297384e-01   1.08389396e+03]
0.0 [-0.00132816  0.03824038  0.77959529] [ -1.60118100e-03   5.12016153e-02   4.01040765e+00]
This is the average radius of curvature of the anes 690.784554585


  0%|          | 2/1261 [00:01<13:39,  1.54it/s]

908.531036186
0.0 [ -1.51695591e-04   2.13000435e-01   2.10907497e+02] [ -1.98663725e-04   3.02490291e-01   1.07937602e+03]
0.0 [-0.00129317  0.03782888  0.78035774] [ -1.69356852e-03   5.37222756e-02   3.99369128e+00]
This is the average radius of curvature of the anes 683.989259053


...

100%|█████████▉| 1259/1261 [15:35<00:01,  1.04it/s]

914.416359083
0.0 [  1.23154816e-05  -5.22248145e-02   3.36821764e+02] [ -4.59038237e-05   3.02934553e-02   1.22199841e+03]
0.0 [  1.04987018e-04  -9.27512705e-03   1.24624053e+00] [ -3.91320916e-04   5.38011766e-03   4.52139411e+00]
This is the average radius of curvature of the anes 6040.88732477


100%|█████████▉| 1260/1261 [15:36<00:00,  1.13it/s]

918.66613832
0.0 [  1.12386333e-05  -4.59503689e-02   3.30324383e+02] [ -3.84058732e-05   3.68224516e-02   1.21514114e+03]
0.0 [  9.58071008e-05  -8.16078551e-03   1.22220022e+00] [ -3.27402388e-04   6.53966741e-03   4.49602221e+00]
This is the average radius of curvature of the anes 6746.61191418


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video.mp4 

CPU times: user 12min 12s, sys: 1min 2s, total: 13min 15s
Wall time: 15min 39s
